In [1]:
import torch
from PIL import Image
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.utils as vutils
import os
import glob
import datetime
import time
start_time = datetime.datetime.now()
print(start_time)

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

2025-03-29 14:40:04.756013


In [2]:
device = torch.device("cuda:3")
print(torch.cuda.is_available())

True


In [3]:
# state_dict
model_path_hmvae = './saved_models_hmvae_mnist/model_epoch_496_loss_21.7908.pt'
state_dict = torch.load(model_path_hmvae, map_location=device)
model_path_MEMvae = './saved_models_MEMvae_mnist/model_epoch_498_loss_22.3760.pt'
state_dict_MEMvae = torch.load(model_path_MEMvae, map_location=device)
model_path_vae = './saved_models_vae_mnist/model_epoch_492_loss_23.7127.pt'
state_dict_vae = torch.load(model_path_vae, map_location=device)

In [ ]:
from torchvision.transforms import ToTensor


mnist_test = MNIST(root="./mnistdata", train=False, download=True)
real_images_dir = "./mnist_real_images"
os.makedirs(real_images_dir, exist_ok=True)


for i, (img_pil, label) in enumerate(mnist_test):
    img_pil.save(os.path.join(real_images_dir, f"image_{i:04d}.png"))

In [ ]:
from HMVAE import HMVAE
n_levels = 3
n_regions=2
region_slot_config= [
    ( [50, 50, 50], [64, 128, 256] ),  
    ( [64, 64,64],   [32,64, 128] )       
]
latent_dim = 20
hidden_dim =128
image_channels = 1  # MNIST has 1 channel
image_size = 28  # 28x28 images
model_hmvae = HMVAE( n_levels, n_regions, region_slot_config,latent_dim,hidden_dim,image_channels,image_size).to(device)
model_hmvae.load_state_dict(state_dict)
model_hmvae.to(device)

HMVAE(
  (encoder): Encoder(
    (linears): Sequential(
      (0): Linear(in_features=784, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=256, out_features=128, bias=True)
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (latent): Linear(in_features=128, out_features=20, bias=True)
    (get_mu): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=20, bias=True)
    )
    (get_logvar): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=20,

In [6]:
#Loading 1-layers
from MEMVAE import MEMVAE
n_levels = 1
num_slots_list = [50]  # Just example sizes
slot_dim_list = [64]  # Example embedding sizes
latent_dim = 20
hidden_dim =128
image_channels = 1  # MNIST has 1 channel
image_size = 28  # 28x28 images
model_MEMvae= MEMVAE( n_levels, num_slots_list, slot_dim_list,latent_dim,hidden_dim,image_channels,image_size).to(device)
model_MEMvae.load_state_dict(state_dict_MEMvae)
model_MEMvae.to(device)

MEMVAE(
  (encoder): Encoder(
    (linears): Sequential(
      (0): Linear(in_features=784, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=256, out_features=128, bias=True)
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (latent): Linear(in_features=128, out_features=20, bias=True)
    (get_mu): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=20, bias=True)
    )
    (get_logvar): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=20

In [7]:
from VAE import VAE
model_vae  = VAE(latent_dim=20,hidden_dim=128,image_channels=1,image_size=28).to(device)
model_vae.load_state_dict(state_dict_vae)

<All keys matched successfully>

In [ ]:
def generate_from_latent(model, num_samples, save_dir, latent_dim=20):
    model.eval()
    
    os.makedirs(save_dir, exist_ok=True)
    with torch.no_grad():
      
        z = torch.randn(num_samples, latent_dim).to(device)
        
        if model == model_vae:
            gen_data = model.decoder(z)  
        else:
            gen_data, _ = model.decoder(z)  
            
        gen_data = gen_data.view(-1, 1, 28, 28)
        
       
        for i in range(num_samples):
            img_path = os.path.join(save_dir, f'image_{i:04d}.png')  
            vutils.save_image(gen_data[i], img_path, normalize=False)  


generate_from_latent(model_vae, num_samples=10000, save_dir='./generated_images_vae')
generate_from_latent(model_hmvae, num_samples=10000, save_dir='./generated_images_hmvae')
generate_from_latent(model_MEMvae, num_samples=10000, save_dir='./generated_images_MEMVAE')

In [ ]:
import random 
from torchvision.utils import make_grid, save_image
models = ["vae", "hmvae", "MEMVAE"]


output_dir = "./generated_grids"
os.makedirs(output_dir, exist_ok=True)

for model in models:
    input_folder = f"./generated_images_{model}"
    
    
    if not os.path.exists(input_folder):
        print(f"警告：{input_folder} 不存在，跳过")
        continue
    
    
    all_images = glob.glob(os.path.join(input_folder, "*.png"))  
    if len(all_images) < 64:
        print(f"警告：{input_folder} 中图像不足64张（实际有{len(all_images)}张），跳过")
        continue
    
   
    image_paths = random.sample(all_images, 64)  
    
  
    tensor_list = []
    transform = transforms.ToTensor()
    for img_path in image_paths:
        img = Image.open(img_path).convert("RGB")
        tensor = transform(img).unsqueeze(0)  
        tensor_list.append(tensor)
    
   
    batch = torch.cat(tensor_list, dim=0)
    grid = make_grid(batch, nrow=8, padding=2, normalize=True)
    
    
    output_path = os.path.join(output_dir, f"{model}_grid_8x8.png")
    save_image(grid, output_path)
    print(f"{model} 的8x8网格图已保存到：{output_path}") 

vae 的8x8网格图已保存到：./generated_grids\vae_grid_8x8.png
hmvae 的8x8网格图已保存到：./generated_grids\hmvae_grid_8x8.png
MEMVAE 的8x8网格图已保存到：./generated_grids\MEMVAE_grid_8x8.png


In [ ]:
from PIL import Image
def convert_grayscale_to_rgb(image_path):
    img = Image.open(image_path).convert("L")  
    img_rgb = np.stack([np.array(img)] * 3, axis=-1)  
    Image.fromarray(img_rgb).save(image_path)  

In [ ]:
for img_path in glob.glob("./generated_images_vae/*.png"):
    convert_grayscale_to_rgb(img_path)
for img_path in glob.glob("./generated_images_hmvae/*.png"):
    convert_grayscale_to_rgb(img_path)
for img_path in glob.glob("./generated_images_MEMVAE/*.png"):
    convert_grayscale_to_rgb(img_path)

for img_path in glob.glob("./mnist_real_images/*.png"):
    convert_grayscale_to_rgb(img_path)

In [13]:
import subprocess
models = [
    "vae", 
    "hmvae", 
    "MEMVAE"
]
for model in models:
    gen_dir = f"./generated_images_{model}"
    cmd = f"python -m pytorch_fid  {gen_dir} ./mnist_real_images"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    print(f"FID for {model}: {result.stdout}")

FID for vae: FID:  44.57462378884898

FID for hmvae: FID:  27.84932459097979

FID for MEMVAE: FID:  32.1096883541494

